In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
EPOCHS = 300
BATCH_SIZE = 128

In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else 'cpu')

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        root = './.data',
        train = True,
        transform=transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),            
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))            
        ])        
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        root = './.data',
        train = False,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])        
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [5]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        
        return out

In [6]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        
        self.layer1 = self._make_layer(16, 2, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.linear = nn.Linear(64, num_classes)
        
    def _make_layer(self, planes, num_blockes, stride):
        strides = [stride] + [1]*(num_blockes-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes

        return nn.Sequential(*layers)    
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        
        return out

In [7]:
model = ResNet().to(DEVICE)

In [8]:
# optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005)
optimizer = optim.Adam(model.parameters(), lr=0.1, weight_decay=0.0005)


In [9]:
schedular = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [10]:
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [11]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        
        optimizer.zero_grad()
        output = model(data)
        
        loss = F.cross_entropy(output, target)
        loss.backward()
        
        optimizer.step()

In [12]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.0 * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [ ]:
for epoch in range(1, EPOCHS+1):
    schedular.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(f'{epoch} Test Loss : {test_loss:.4f}, Accuracy : {test_accuracy:.2f}')

d:\sources\practice-deeplearning-pytorch\.env\lib\site-packages\torch\optim\lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


1 Test Loss : 3.4348, Accuracy : 11.22
2 Test Loss : 1.9181, Accuracy : 24.06
3 Test Loss : 2.7358, Accuracy : 16.65
4 Test Loss : 1.8221, Accuracy : 28.77
5 Test Loss : 3.6765, Accuracy : 14.76
6 Test Loss : 2.0836, Accuracy : 26.04
7 Test Loss : 1.7955, Accuracy : 30.71
8 Test Loss : 1.7417, Accuracy : 31.00
9 Test Loss : 6.6151, Accuracy : 10.21
10 Test Loss : 2.2436, Accuracy : 20.03
11 Test Loss : 2.9425, Accuracy : 20.43
12 Test Loss : 2.1001, Accuracy : 26.09
13 Test Loss : 9.3518, Accuracy : 13.44
14 Test Loss : 2.7298, Accuracy : 14.33
15 Test Loss : 2.7180, Accuracy : 20.54
16 Test Loss : 2.9923, Accuracy : 18.80
17 Test Loss : 3.3497, Accuracy : 16.73
18 Test Loss : 2.0136, Accuracy : 25.61
19 Test Loss : 1.7866, Accuracy : 31.33
20 Test Loss : 8.8525, Accuracy : 10.06
21 Test Loss : 2.1533, Accuracy : 27.39
22 Test Loss : 10.5010, Accuracy : 10.65
23 Test Loss : 2.0853, Accuracy : 20.57
24 Test Loss : 3.0689, Accuracy : 15.84
25 Test Loss : 2.1220, Accuracy : 24.89
26 Test 